In [2]:
!pip install boto3

     ------------------------------------ 139.3/139.3 kB 485.3 kB/s eta 0:00:00
     -------------------------------------- 82.1/82.1 kB 513.5 kB/s eta 0:00:00
     ---------------------------------------- 11.9/11.9 MB 3.6 MB/s eta 0:00:00


In [10]:
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.client import Config


s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket_name = "d2b-internal-assessment-bucket"

# List all objects in the bucket
response = s3.list_objects(Bucket=bucket_name, Prefix="orders_data")

# Check if there are any objects in the response
if 'Contents' in response:
    # Loop through each object and print its key
    for obj in response['Contents']:
        print("File:", obj['Key'])

File: orders_data/
File: orders_data/analytics_export/faitusie8037/agg_public_holiday.csv
File: orders_data/analytics_export/faitusie8037/agg_shipments.csv
File: orders_data/analytics_export/faitusie8037/best_performing_product.csv
File: orders_data/analytics_export/murtodun9658/agg_public_holiday.csv
File: orders_data/analytics_export/murtodun9658/agg_shipments.csv
File: orders_data/analytics_export/murtodun9658/best_performing_product.csv
File: orders_data/analytics_export/murtodun9658/late_shipments.csv
File: orders_data/analytics_export/murtodun9658/undelivered_shipments.csv
File: orders_data/analytics_export/salimuti8986/agg_public_holiday.csv
File: orders_data/analytics_export/salimuti8986/agg_shipments.csv
File: orders_data/analytics_export/salimuti8986/best_performing_product.csv
File: orders_data/orders.csv
File: orders_data/reviews.csv
File: orders_data/shipment_deliveries.csv


In [8]:
# Download the orders.csv
s3.download_file(bucket_name, "orders_data/orders.csv", "orders.csv")

# Download the reviews.csv
s3.download_file(bucket_name, "orders_data/reviews.csv", "reviews.csv")

# Download the orders.csv
s3.download_file(bucket_name, "orders_data/shipment_deliveries.csv", "shipment_deliveries.csv")

In [9]:
fact_orders = pd.read_csv('orders.csv')

NameError: name 'pd' is not defined